In [2]:
import os

In [3]:
%pwd

'd:\\A_Category\\iNeuron\\End-To-End-NLP-Project-News-Article-Sorting\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\A_Category\\iNeuron\\End-To-End-NLP-Project-News-Article-Sorting'

In [14]:
#DataIngestionConfig goes to entity  
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [15]:
from ArticleSorting.constants import *
from ArticleSorting.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(
                self, 
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir= config.root_dir,
                source_URL=config.source_URL,
                local_data_file= config.local_data_file,
                unzip_dir= config.unzip_dir
          )
          return data_ingestion_config

In [17]:
import os
import urllib.request as request
import zipfile
from ArticleSorting.logging import logger
from ArticleSorting.utils.common import get_size

In [18]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

 
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [20]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-31 20:08:32,747:  INFO: common: yaml file:config\config.yaml loaded successfully]
[2023-07-31 20:08:32,752:  INFO: common: yaml file:params.yaml loaded successfully]
[2023-07-31 20:08:32,755:  INFO: common: created directory at : artifacts]
[2023-07-31 20:08:32,758:  INFO: common: created directory at : artifacts/data_ingestion]
[2023-07-31 20:08:36,538:  INFO: 4261578779: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 1936538
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "0e68aba45962a5084dbbf6e3f95d94aa88116ae0c41f3aaeedf0a4e813d874b3"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 92E2:119C:66542:72362:64C7C061
Accept-Ranges: bytes
Date: Mon, 31 Jul 2023 14:08:33 GMT
Via: 1.1 varnish
X-Served-By: cache-qpg1233-QPG
X-Cache: M